## Import & Configuration

In [ ]:
!pip install ccxt -q

import ccxt
import pandas as pd
import time
import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import skew, kurtosis
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

EXCHANGE_ID = 'bitstamp'
SYMBOL = 'XRP/USD'
TIMEFRAME = '1h'
YEARS_BACK  = 6
RETRY_LIMIT = 3
SLEEP_TIME  = 0.5

print("Import & Configuration Ready.")

---
## DATA ENGINEERING
---

## Miner

In [ ]:
def init_exchange(exchange_id):
    try:
        exchange_class = getattr(ccxt, exchange_id)
        exchange = exchange_class({
            'enableRateLimit': True,
        })
        return exchange
    except AttributeError:
        print(f"❌ Exchange '{exchange_id}' not found.")
        return None

def fetch_historical_data_robust(symbol, timeframe, years, exchange):
    now = exchange.milliseconds()
    start_time = now - (years * 365 * 24 * 60 * 60 * 1000)

    print(f"🔄 Target Mining: {pd.to_datetime(start_time, unit='ms')} until Now")
    print(f"   Exchange: {exchange.id.upper()}")

    all_ohlcv = []
    current_since = start_time

    while current_since < now:
        # --- LOGIC RETRY ---
        success = False
        ohlcv = []

        for attempt in range(RETRY_LIMIT):
            try:
                ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since=current_since, limit=1000)
                success = True
                break
            except Exception as e:
                print(f"⚠️ Error (Attempt {attempt+1}/{RETRY_LIMIT}): {e}")
                time.sleep(2 * (attempt + 1))

        if not success:
            print("❌ Complete failure after several attempts. Stopping.")
            break

        if not ohlcv:
            print("⚠️ Warning: Exchange stopped giving data (EOF).")
            break

        # --- SANITY CHECK ---
        first_candle_time = ohlcv[0][0]
        time_diff_days = (first_candle_time - current_since) / (1000 * 60 * 60 * 24)

        if time_diff_days > 30 and len(all_ohlcv) == 0:
            print(f"❌ CRITICAL: Exchange ignores history parameters.")
            print(f"   Diminta: {pd.to_datetime(current_since, unit='ms')}")
            print(f"   Dikasih: {pd.to_datetime(first_candle_time, unit='ms')}")
            return []

        all_ohlcv.extend(ohlcv)

        # Update pointer
        last_timestamp = ohlcv[-1][0]
        last_date_human = pd.to_datetime(last_timestamp, unit='ms')
        print(f"   -> Progress: {last_date_human} | Total: {len(all_ohlcv)} rows")

        if last_timestamp >= current_since:
             current_since = last_timestamp + 1
        else:
             current_since += (1000 * 60 * 60)

        time.sleep(SLEEP_TIME)

    return all_ohlcv

print("Starting Data Mining...")
exchange = init_exchange(EXCHANGE_ID)

if exchange:
    raw_data = fetch_historical_data_robust(SYMBOL, TIMEFRAME, YEARS_BACK, exchange)
    print(f"Done! Collected {len(raw_data)} raw candles.")
else:
    print("Failed to initialize exchange.")
    raw_data = []

## Validation & Storage

In [ ]:
def process_and_validate(raw_data_list):
    if not raw_data_list:
        print("Empty Data/Empty List. Cek the output of 2nd Cell.")
        return None

    print("\nCleaning Started...")

    # Convert ke DataFrame
    df = pd.DataFrame(raw_data_list, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

    # Convert Timestamp & Set Index
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)

    # Deduplicate
    initial_len = len(df)
    df = df[~df.index.duplicated(keep='first')]
    dedup_len = len(df)

    if initial_len != dedup_len:
        print(f"   -> Founded & Deleted {initial_len - dedup_len} Duplicated Row.")
    else:
        print("   -> No Duplicate. Clean data.")

    df.sort_index(inplace=True)

    print("\n--- DATA QUALITY REPORT ---")
    print(f"Range Data  : {df.index.min()} until {df.index.max()}")
    print(f"Total Duration: {(df.index.max() - df.index.min()).days} Days")
    print(f"Total Row : {len(df)}")

    missing = df.isnull().sum().sum()
    if missing > 0:
        print(f"⚠️ WARNING: There's {missing} NaN!")
    else:
        print("Complete Data (No Missing Values).")

    return df

if 'raw_data' in locals() and raw_data:
    df_engine = process_and_validate(raw_data)

    if df_engine is not None:
        filename = f"raw_{SYMBOL.replace('/', '')}_{TIMEFRAME}.csv"
        df_engine.to_csv(filename)
        print(f"\nSUCCESS. Dataset saved: {filename}")
else:
    print("'raw_data' Variable not found or null. Run the 2nd Cell first!")

---
## DATA ANALYSIS
---

## Load

In [ ]:
filename = f"raw_{SYMBOL.replace('/', '')}_{TIMEFRAME}.csv"

try:
    df = pd.read_csv(filename)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.set_index('timestamp', inplace=True)

    print("Data Loaded Successfully!")
    print(f"Periode: {df.index.min()} s/d {df.index.max()}")
    print(f"Total Duration: {(df.index.max() - df.index.min()).days} Days")
    print(f"Total Row: {len(df)}")

    print(df.info())

except FileNotFoundError:
    print(f"There is no file named {filename}. Run the Data Engineering first.")

## EDA

In [ ]:
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (20, 30)
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14

# --- A. DATA PREPARATION ---
eda_df = df.copy()

# 1. Basic Metrics
eda_df['log_return'] = np.log(eda_df['close'] / eda_df['close'].shift(1))
eda_df['volatility_30d'] = eda_df['log_return'].rolling(window=24*30).std() * np.sqrt(24*365) # Annualized Vol
eda_df['cum_return'] = (1 + eda_df['log_return']).cumprod()

# 2. Time Features
eda_df['hour'] = eda_df.index.hour
eda_df['day_name'] = eda_df.index.day_name()
eda_df['month'] = eda_df.index.month
eda_df['year'] = eda_df.index.year

# 3. Drawdown Analysis
eda_df['running_max'] = eda_df['close'].cummax()
eda_df['drawdown'] = (eda_df['close'] - eda_df['running_max']) / eda_df['running_max']

# 4. Clean Data
eda_df.dropna(inplace=True)

# 5. Statistical Tests
adf_result = adfuller(eda_df['close'])
adf_pvalue = adf_result[1]
adf_status = "STATIONARY" if adf_pvalue < 0.05 else "NON-STATIONARY (Dangerous for ML)"

# --- B. DASHBOARD PLOTTING ---
fig = plt.figure(constrained_layout=True)
gs = fig.add_gridspec(6, 2)

# PANEL 1: Price vs Volatility (Macro View)
ax1 = fig.add_subplot(gs[0, :])
color_price = '#1f77b4' # Bloomberg Blue
ax1.plot(eda_df.index, eda_df['close'], color=color_price, alpha=0.8, label='Price', linewidth=1)
ax1.set_ylabel('Price (USD)', color=color_price, fontweight='bold')
ax1.tick_params(axis='y', labelcolor=color_price)
ax1.set_title(f'1. Macro View: Price Action vs Market Fear (Volatility)', fontweight='bold')
ax1.grid(True, which='major', linestyle='--', alpha=0.5)

ax1_twin = ax1.twinx()
color_vol = '#d62728' # Alert Red
ax1_twin.set_ylabel('Annualized Volatility (30D)', color=color_vol, fontweight='bold')
ax1_twin.plot(eda_df.index, eda_df['volatility_30d'], color=color_vol, alpha=0.3, linestyle='-', label='Volatility', linewidth=1)
ax1_twin.tick_params(axis='y', labelcolor=color_vol)
ax1_twin.fill_between(eda_df.index, 0, eda_df['volatility_30d'], color=color_vol, alpha=0.05)

# PANEL 2: Equity Curve
ax2 = fig.add_subplot(gs[1, 0])
ax2.plot(eda_df.index, eda_df['cum_return'], color='#2ca02c', linewidth=1.5) # Profit Green
ax2.set_title(f'2. Growth of $1 Investment (Total Return: {(eda_df["cum_return"].iloc[-1]-1)*100:.2f}%)', fontweight='bold')
ax2.fill_between(eda_df.index, 1, eda_df['cum_return'], alpha=0.1, color='#2ca02c')
ax2.grid(True, linestyle='--', alpha=0.5)

# PANEL 3: Underwater Plot
ax3 = fig.add_subplot(gs[1, 1])
ax3.plot(eda_df.index, eda_df['drawdown'], color='#d62728', linewidth=1)
ax3.fill_between(eda_df.index, 0, eda_df['drawdown'], color='#d62728', alpha=0.2)
ax3.set_title(f'3. Underwater Plot (Max Drawdown: {eda_df["drawdown"].min()*100:.2f}%)', fontweight='bold')
ax3.set_ylabel('Drawdown (%)')
ax3.grid(True, linestyle='--', alpha=0.5)

# PANEL 4: Monthly Seasonality
ax4 = fig.add_subplot(gs[2, 0])
monthly_returns = eda_df.groupby(['year', 'month'])['log_return'].sum().unstack()
month_map = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}
monthly_returns.columns = monthly_returns.columns.map(month_map)
sns.heatmap(monthly_returns, annot=True, fmt='.1%', cmap='RdYlGn', center=0, ax=ax4, cbar=False, annot_kws={"size": 9})
ax4.set_title('4. Monthly Seasonality (Calendar Heatmap)', fontweight='bold')

# PANEL 5: Hourly Micro-Structure
ax5 = fig.add_subplot(gs[2, 1])
hourly_pivot = eda_df.groupby(['day_name', 'hour'])['log_return'].mean().unstack()
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
hourly_pivot = hourly_pivot.reindex(days_order)
sns.heatmap(hourly_pivot, cmap='RdYlGn', center=0, ax=ax5, cbar_kws={'label': 'Avg Return'})
ax5.set_title('5. Intraday Strategy: Best Time to Trade', fontweight='bold')
ax5.set_ylabel('')

# PANEL 6: Fat Tail Analysis
ax6 = fig.add_subplot(gs[3, 0])
sns.histplot(eda_df['log_return'], bins=100, kde=True, color='#9467bd', ax=ax6, stat="density", element="step")
mu, std = eda_df['log_return'].mean(), eda_df['log_return'].std()
x = np.linspace(mu - 4*std, mu + 4*std, 100)
p = (1 / (np.sqrt(2 * np.pi) * std)) * np.exp(-0.5 * ((x - mu) / std)**2)
ax6.plot(x, p, 'k--', linewidth=2, label='Normal Dist (Theory)')
ax6.set_title(f'6. Risk Distribution (Kurtosis: {kurtosis(eda_df["log_return"]):.2f})', fontweight='bold')
ax6.set_xlim(-0.05, 0.05)
ax6.legend()
ax6.grid(True, linestyle='--', alpha=0.5)

# PANEL 7: Autocorrelation
ax7 = fig.add_subplot(gs[3, 1])
pd.plotting.autocorrelation_plot(eda_df['log_return'].resample('1D').mean(), ax=ax7)
ax7.set_title('7. Market Efficiency Test (Autocorrelation)', fontweight='bold')
ax7.set_xlim(0, 30)
ax7.grid(True, linestyle='--', alpha=0.5)

# PANEL 8: Price-Volume Correlation Rolling
ax8 = fig.add_subplot(gs[4, 0])
rolling_corr = eda_df['close'].rolling(window=24*30).corr(eda_df['volume'])
ax8.plot(eda_df.index, rolling_corr, color='#ff7f0e', linewidth=1)
ax8.axhline(0, color='black', linestyle='--', linewidth=1)
ax8.set_title('8. Trend Validation: Price-Volume Correlation (30D Rolling)', fontweight='bold')
ax8.set_ylabel('Correlation')
ax8.fill_between(eda_df.index, 0, rolling_corr, where=(rolling_corr>0), color='green', alpha=0.1)
ax8.fill_between(eda_df.index, 0, rolling_corr, where=(rolling_corr<0), color='red', alpha=0.1)
ax8.grid(True, linestyle='--', alpha=0.5)

# PANEL 9: Risk per Day (Bar Chart)
ax9 = fig.add_subplot(gs[4, 1])
daily_vol = eda_df.groupby('day_name')['log_return'].std() * np.sqrt(24) * 100
daily_vol = daily_vol.reindex(days_order)
sns.barplot(x=daily_vol.index, y=daily_vol.values, ax=ax9, palette='viridis')
ax9.set_title('9. Daily Volatility Profile (Risk by Day)', fontweight='bold')
ax9.set_ylabel('Avg Daily Vol (%)')
ax9.set_xticklabels(days_order, rotation=45)
ax9.grid(True, axis='y', linestyle='--', alpha=0.5)

# PANEL 10: Outlier Detection (Boxplot)
ax10 = fig.add_subplot(gs[5, :])
sns.boxplot(x='day_name', y='log_return', data=eda_df, order=days_order, ax=ax10, palette='Set2', fliersize=1)
ax10.set_title('10. Weekly Anomaly Detection (Outliers)', fontweight='bold')
ax10.set_ylim(-0.05, 0.05)
ax10.grid(True, axis='y', linestyle='--', alpha=0.5)

plt.suptitle(f'ADF Status: {adf_status} (p={adf_pvalue:.4f})', fontsize=24, y=1.01, fontweight='bold')
plt.show()

# --- D. TEXT SUMMARY ---
print("\n📊 --- EXECUTIVE QUANTITATIVE REPORT ---")
print(f"1. MARKET REGIME")
print(f"   - Volatility Trend      : {'Increasing' if eda_df['volatility_30d'].iloc[-1] > eda_df['volatility_30d'].mean() else 'Decreasing'}")
print(f"   - Volume Confirmation   : {rolling_corr.iloc[-1]:.2f} (Correlation)")
print(f"     (Positif = Healthy Trend , Negative/Zero = Weak Trend/Divergen)")

print(f"\n2. RISK CALENDAR")
print(f"   - Riskiest Day          : {daily_vol.idxmax()} (Vol: {daily_vol.max():.2f}%)")
print(f"   - Safest Day            : {daily_vol.idxmin()} (Vol: {daily_vol.min():.2f}%)")
print(f"   - Best Month            : {monthly_returns.mean().idxmax()}")

print(f"\n3. MODELING INSIGHTS")
print(f"   - Stationarity          : {adf_status} (Needs Differencing/Log Return)")
print(f"   - Fat Tails             : Kurtosis {kurtosis(eda_df['log_return']):.2f} -> Needs Robust Scaling")

---
## DATA SCIENCE
---

## Preprocessing

In [ ]:
def technical_indicators_engine(df):
    data = df.copy()

    # 1. TREND & MOMENTUM (Standard Indicators)
    data['ema_50']  = data['close'].ewm(span=50, adjust=False).mean()
    data['ema_200'] = data['close'].ewm(span=200, adjust=False).mean()
    data['dist_ema_50']  = (data['close'] - data['ema_50']) / data['ema_50']
    data['dist_ema_200'] = (data['close'] - data['ema_200']) / data['ema_200']

    delta = data['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    data['rsi'] = 100 - (100 / (1 + rs))

    min_rsi = data['rsi'].rolling(window=14).min()
    max_rsi = data['rsi'].rolling(window=14).max()
    data['stoch_rsi'] = (data['rsi'] - min_rsi) / (max_rsi - min_rsi)

    exp12 = data['close'].ewm(span=12, adjust=False).mean()
    exp26 = data['close'].ewm(span=26, adjust=False).mean()
    data['macd'] = exp12 - exp26
    data['macd_signal'] = data['macd'].ewm(span=9, adjust=False).mean()
    data['macd_hist'] = data['macd'] - data['macd_signal']

    data['bb_middle'] = data['close'].rolling(window=20).mean()
    data['bb_std']    = data['close'].rolling(window=20).std()
    data['bb_upper']  = data['bb_middle'] + (data['bb_std'] * 2)
    data['bb_lower']  = data['bb_middle'] - (data['bb_std'] * 2)
    data['bb_percent'] = (data['close'] - data['bb_lower']) / (data['bb_upper'] - data['bb_lower'])

    high_low   = data['high'] - data['low']
    high_close = np.abs(data['high'] - data['close'].shift())
    low_close  = np.abs(data['low'] - data['close'].shift())
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    data['atr'] = tr.rolling(window=14).mean()
    data['atr_pct'] = data['atr'] / data['close']

    data['vol_change'] = data['volume'].pct_change()
    data['vol_ma_20'] = data['volume'].rolling(window=20).mean()
    data['vol_ratio'] = data['volume'] / data['vol_ma_20']

    # --- HALVING AWARENESS ---
    halving_dates = [
        pd.Timestamp('2016-07-09'),
        pd.Timestamp('2020-05-11'),
        pd.Timestamp('2024-04-20'),
        pd.Timestamp('2028-04-17')
    ]

    def get_days_since_halving(current_date):
        past_halvings = [date for date in halving_dates if date <= current_date]
        if not past_halvings:
            # Fallback if the data older than 2016
            return 0
        last_halving = past_halvings[-1]
        return (current_date - last_halving).days

    data['days_since_halving'] = data.index.to_series().apply(get_days_since_halving)

    # Normalize: The halving cycle is ~1460 days. Divide it so the number is 0.0 - 1.0
    data['halving_progress'] = data['days_since_halving'] / 1460.0

    # --- TARGET ENGINEERING ---
    data['log_return'] = np.log(data['close'] / data['close'].shift(1))
    data['target_return'] = data['log_return'].shift(-1)

    # Dynamic Threshold (Smart Target)
    data['dynamic_threshold'] = data['atr_pct'].rolling(window=24).mean() * 0.5
    data['dynamic_threshold'].fillna(0.002, inplace=True)

    data['target_class'] = np.where(data['target_return'] > data['dynamic_threshold'], 1, 0)

    # CLEANUP
    data.dropna(inplace=True)
    data.replace([np.inf, -np.inf], 0, inplace=True)

    return data

# APPLY
print("Starting Feature Engineering...")
df_processed = technical_indicators_engine(df)
print(f"DONE!")
print(df_processed[['close', 'days_since_halving', 'halving_progress', 'target_class']].tail())

## Modelling

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit

# 1. SETUP DATASET
drop_cols = ['target_class', 'target_return', 'log_return', 'open', 'high', 'low', 'close', 'dynamic_threshold']
X = df_processed.drop(columns=drop_cols, errors='ignore')
y = df_processed['target_class']

# 2. TIME SERIES SPLIT
train_size = int(len(X) * 0.8)
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

print(f"Training Data: {len(X_train)} | Testing Data: {len(X_test)}")

# 3. HYPERPARAMETER TUNING
param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 15, 20, None],
    'min_samples_split': [10, 20, 50],
    'min_samples_leaf': [5, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'class_weight': ['balanced', 'balanced_subsample']
}

# Model init
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Time Series Cross-Validation
tscv = TimeSeriesSplit(n_splits=3)

print("\nStarting Hyperparameter Tuning...")

# RandomizedSearchCV finding 10 random best combination
search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=10,
    scoring='precision',
    cv=tscv,
    verbose=3,
    random_state=42,
    n_jobs=-1
)

search.fit(X_train, y_train)

best_model = search.best_estimator_
print(f"\nDone!")
print(f"Best Settings: {search.best_params_}")

# 4. EVALUATE
print("\n--- Best Model Perform (Data Test) ---")
preds = best_model.predict(X_test)

acc = accuracy_score(y_test, preds)
print(f"Accuracy: {acc:.2%}")
print("\nClassification Report:")
print(classification_report(y_test, preds))

# Confusion Matrix
cm = confusion_matrix(y_test, preds)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', cbar=False)
plt.xlabel('AI Predict')
plt.ylabel('Reality')
plt.title('Confusion Matrix (Optimized Model)')
plt.show()

# Feature Importance
importances = best_model.feature_importances_
indices = np.argsort(importances)[::-1]
top_n = 15

plt.figure(figsize=(10, 8))
plt.title("Decision-Making Features (Optimized)")
plt.barh(range(top_n), importances[indices[:top_n]], align="center", color='#2ca02c')
plt.yticks(range(top_n), [X.columns[i] for i in indices[:top_n]])
plt.gca().invert_yaxis()
plt.show()

# Update variabel global 'model'
model = best_model

## Prediction

In [ ]:
def generate_narrative(row, signal):
    reasons = []

    # Best Value
    rsi = row['rsi'].values[0]
    macd_hist = row['macd_hist'].values[0]
    vol_ratio = row['vol_ratio'].values[0]
    ema_dist = row['dist_ema_200'].values[0]
    bb_pos = row['bb_percent'].values[0]

    if signal == 1: # RECOMMENDATION BUY
        if rsi < 30: reasons.append("RSI in Oversold territory; high probability of a technical rebound.")
        elif rsi > 50 and rsi < 70: reasons.append("Strong bullish RSI momentum; significant upside remains before Overbought.")

        if macd_hist > 0: reasons.append("Positive MACD Histogram confirms upward momentum.")
        if vol_ratio > 1.0: reasons.append(f"Volume is {vol_ratio:.1f}x above average, validating move strength.")
        if ema_dist > 0: reasons.append("Price holding above EMA 200; Primary Trend is Bullish.")
        if bb_pos < 0.2: reasons.append("Price testing Lower Bollinger Band; potential mean reversion long.")

    else: # RECOMMENDATION WAIT/HOLD
        if rsi > 70: reasons.append("RSI Overbought; high correction risk.")
        if macd_hist < 0: reasons.append("Negative MACD Histogram; momentum is fading.")
        if ema_dist < 0: reasons.append("Price below EMA 200; Primary Trend is Bearish.")
        if bb_pos > 0.8: reasons.append("Price near Upper Bollinger Band; asset is overextended.")
        if vol_ratio < 0.8: reasons.append("Anemic volume; lack of market conviction.")

    if not reasons:
        reasons.append("Mixed technical signals. No clear edge detected.")

    return reasons

def generate_signal_with_reasoning(model, df_processed, feature_cols):
    latest_data = df_processed.iloc[[-1]].copy()
    input_features = latest_data[feature_cols]

    prediction = model.predict(input_features)[0]
    probabilities = model.predict_proba(input_features)[0]
    confidence = probabilities[prediction]

    # Feature Importance
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]

    # Top 3 Features that most influence the model
    top_drivers = []
    for i in indices[:3]:
        feature_name = feature_cols[i]
        feature_val = input_features[feature_name].values[0]
        top_drivers.append((feature_name, feature_val))

    return latest_data, prediction, confidence, top_drivers

feature_columns = X_train.columns
latest_row, signal, conf, top_drivers = generate_signal_with_reasoning(model, df_processed, feature_columns)

current_price = latest_row['close'].values[0]
timestamp     = latest_row.index[0]
rsi_now       = latest_row['rsi'].values[0]

print("--- REPORT ---")
print(f"Time (UTC)    : {timestamp}")
print(f"Asset         : {SYMBOL}")
print(f"Current Price : ${current_price:,.2f}")
print("=" * 60)

# RECOMMENDATION
if signal == 1:
    print(f"SIGNAL     : 🟢 BUY / LONG")
    color_code = "🟢"
else:
    print(f"SIGNAL     : 🔴 WAIT / HOLD (No Entry)")
    color_code = "🔴"

print(f"Confidence : {conf*100:.2f}%")
if conf < 0.6:
    print("Risk Note  : Confidence < 60%. Weak Signal, be carefull.")

print("-" * 60)

# "WHY"
print("BASIS OF DECISION:")

# Narrative Logic
narratives = generate_narrative(latest_row, signal)
for i, reason in enumerate(narratives, 1):
    print(f"   {i}. {reason}")

print("-" * 40)

# Top Technical Drivers
print("KEY INDICATORS WATCHED (Faktor Dominan):")
for name, val in top_drivers:
    print(f"   • {name.ljust(15)} : {val:.4f}")

print("=" * 60)
print("Disclaimer: Statistical probability only. Not financial advice.")